# import libraries

In [1]:
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dropout,Dense
from keras.models import Sequential
from keras.datasets import mnist
from PIL import ImageGrab,Image
from keras import backend as k
import tkinter as tk
from tkinter import *
import keras
import win32gui
import numpy as np

# load data set

In [2]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
print(x_train.shape,y_train.shape,
     x_test.shape,y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


# preprocessing Data

In [3]:
x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)

y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train=x_train/255
x_test=x_test/255
print('x_train shape',x_train.shape)
print(x_train.shape[0],'train samples')
print(x_test.shape[0],'test samples')

x_train shape (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Create the  Model

In [4]:
input_shape=(28,28,1)
batch_size=128
epochs=10
num_classes=10

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))

#model compile

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])




# Train the model

In [5]:
his=model.fit(x_train,y_train,
             batch_size=batch_size,
             epochs=epochs,
             verbose=1,
             validation_data=(x_test,y_test))
print('the model has successfully trained')

Epoch 1/10
469/469 [==============================] - 145s 306ms/step - loss: 0.1995 - accuracy: 0.9396 - val_loss: 0.0441 - val_accuracy: 0.9853
Epoch 2/10
469/469 [==============================] - 143s 304ms/step - loss: 0.0631 - accuracy: 0.9808 - val_loss: 0.0323 - val_accuracy: 0.9888
Epoch 3/10
469/469 [==============================] - 142s 304ms/step - loss: 0.0464 - accuracy: 0.9854 - val_loss: 0.0284 - val_accuracy: 0.9913
Epoch 4/10
469/469 [==============================] - 143s 304ms/step - loss: 0.0370 - accuracy: 0.9883 - val_loss: 0.0256 - val_accuracy: 0.9921
Epoch 5/10
469/469 [==============================] - 142s 303ms/step - loss: 0.0309 - accuracy: 0.9901 - val_loss: 0.0277 - val_accuracy: 0.9910
Epoch 6/10
469/469 [==============================] - 141s 302ms/step - loss: 0.0280 - accuracy: 0.9908 - val_loss: 0.0255 - val_accuracy: 0.9919
Epoch 7/10
469/469 [==============================] - 143s 304ms/step - loss: 0.0233 - accuracy: 0.9926 - val_loss: 0.0294 -

# Evaluate the model

In [9]:
score=model.evaluate(x_test,y_test,verbose=0)
print('loss',score[0])
print('accuracy',score[1])

loss 0.024789858609437943
accuracy 0.9922999739646912


# Create GUI to predict digit

In [26]:
def predict_digit(img):
    #resize img to 28*28 pixel
    img=img.resize((28,28))
    #convert rgb to grayscale
    img=img.convert('L')
    img=np.array(img)
    
    #reshape to support our model input and Normalization
    img=img.reshape(1,28,28,1)
    img=img/255.0
    
    
    #predicting the class
    res=model.predict([img])[0]
    return np.argmax(res),max(res)

class APP(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x=self.y=0
        
        #creating elements
        self.canvas=tk.Canvas(self,width=300,height=300,bg='white',cursor='cross')
        self.label=tk.Label(self,text='predicting...',font=('HALVETIC',48))
        self.classify_btn=tk.Button(self,text='recognise',command= self.classify_handwriting)
        self.clear_btn=tk.Button(self,text='clear',command=self.clear_all)
        
        #Grid structure
        self.canvas.grid(row=0,column=0,pady=2,sticky=W)
        self.label.grid(row=0,column=1,pady=2,padx=2)
        self.classify_btn.grid(row=1,column=1,pady=2,padx=2)
        self.clear_btn.grid(row=1,column=0,pady=2)
        self.canvas.bind('<B1-Motion>',self.draw_lines)
        
    def clear_all(self):
        self.canvas.delete('all')
    
    def classify_handwriting(self):
        HWND=self.canvas.winfo_id()
        rect=win32gui.GetWindowRect(HWND)
        im=ImageGrab.grab(rect)
        
        digit,acc=predict_digit(im)
        self.label.configure(text=str(digit)+','+str(int(acc*100))+'%')
    
    def draw_lines(self,event):
        self.x=event.x
        self.y=event.y
        r=8
        self.canvas.create_oval(self.x-r,self.y-r,self.x+r,self.y+r,fill='black')
        
app=APP()
mainloop()
        

1/1 [==============================] - 0s 25ms/step
